<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/wav2vec2_korean_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformers를 사용한 한국어 wav2vec2 활용에 대한 문서입니다.

test는 zeroth_korean의 testset을 사용했습니다.

TF가 좋지만 다들 torch로 해서 어쩔수없이 torch로 합니다.

# 1. 설치

pip를 통해 설치를 진행합니다.


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

# 2. 라이브러리 불러오기

wav2vec2 기반이라서 별도의 feature extraction을 필요로 하지 않습니다.

In [2]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import torch

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

ds = load_dataset("bingsu/zeroth-korean")

# 3. 전체 데이터 중 Test set만을 이용함

파일 경로를 읽어서 raw signal을 저장합니다.

이어서 전체 test set에 대하여 결과를 result['transcription']에 저장합니다.

기대되는 WER은 4.74% (0.0474) 입니다.

In [8]:
test_ds = ds['test']

In [10]:
def map_to_pred(batch):
    speech_samples = [item['array'] for item in batch['audio']]
    inputs = processor(speech_samples, sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")
    #attention_mask = inputs.attention_mask.to("cuda")
    
    with torch.no_grad():
        #logits = model(input_values, attention_mask=attention_mask).logits
        logits = model(input_values).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16)

  0%|          | 0/29 [00:00<?, ?ba/s]

In [11]:
import evaluate as eval
wer = eval.load("wer")
cer = eval.load("cer")
result_wer = wer.compute(references=result["text"], predictions=result["transcription"])
result_cer = cer.compute(references=result["text"], predictions=result["transcription"])
print(f'WER:{result_wer*100:.2f}%')
print(f'CER:{result_cer*100:.2f}%')
